# This Jupyter Notebook implements a user interface for the previously developed **RT2_Assignment1** package , in particular the one present in the branch **actions** of the repository at : https://github.com/HolyStone95/RT2_Assignment1. 

## The node **go_to_point.py** in the **scripts** folder of the package has been slightly modified in order to receive linear and angular velocities set by two slider inside this notebook, particularly in the module number 3.

# This particular module starts a ros node, initializes variables and implements two buttons in order to start and stop the robot random behaviour

When the button to stop the robot is pressed the variable **cancelled** gets incremented, the client to **/user_interface** sends a **Command** to the server, which handles the cancellation of the previous goal and sets robot velocities to 0. On the other end the button start sends a **Command** to start the robot random behaviour ( Please note that the robot won't actually start moving until the velocities are set in the next module). A subscriber to **/go_to_point/result** increments the value of variable **reached** every time a target position is achieved. For each reached target it also computes the elapsed time to reach it that will be displayed in module number 10.

In [ ]:
from ipywidgets import Button, Layout, ButtonStyle, HBox, GridBox
from rt2_assignment1.srv import Command
from rt2_assignment1.msg import GoalReachingAction
from rt2_assignment1.msg import GoalReachingActionResult
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry
from matplotlib import animation, rc
from matplotlib.animation import FuncAnimation
import actionlib
import ipywidgets as widgets
import matplotlib.pyplot as plt
import rospy
import numpy as np
import jupyros as jr
import time
%matplotlib widget


rospy.init_node('jupyter_node')
## Client to send start and stop commands
ui_client = rospy.ServiceProxy('/user_interface', Command)
## Var to keep track of robot moving or not
moving = False
## Var to keep track of number of cancelled targets
cancelled = 0
## Var to keep track of number of reached targets
reached = 0

## Initializing clock
clock = time.time()
## Var to keep track of time for plots
time_passed=list(range(0,10))
## Var to store elapsed times for reached targets
times = []
## List to store values of linear vel from /cmd_vel
cmd_lin_data=list()
## List to store values of angular vel from /cmd_vel
cmd_ang_data=list()
## List to store values of linear vel from /odom
act_lin_data=list()
## List to store values of angular vel from /odom
act_ang_data=list()

b1 = Button(
    description='Start',
    layout=Layout(width='25%', align="center", grid_area='b1'),
    style=ButtonStyle(button_color='lightblue'))
b2 = Button(description='Stop',
    layout=Layout(width='25%', grid_area='b2'),
    style=ButtonStyle(button_color='moccasin'))

## Callback linked to /go_to_point/result
#
#  Updates a variable to count how many reached targets 
#  and saves times
# @var reached keeps track of number of reached targets
# @var times stores the elapsed time for each reached target
def clbk(msg):
    global reached,stop_time, start_time
    stop_time=time.time()
    reached = reached+1
    times.append(stop_time - start_time)

## Function triggered when the start button is pressed
#
# @var moving keeps track of if robot is moving or not
def on_button1_clicked(b):
    global moving, start_time
    print("Robot random behaviour starting")
    ui_client("start")
    start_time = time.time()
    moving = True
    
## Function triggered when the stop button is pressed
#
# Updates the variable keeping track of cancelled targets
# @var moving keeps track of if robot is moving or not   
# @var cancelld keeps track of number of cancelled goals
def on_button2_clicked(b):
    global moving, cancelled
    print("Robot random behaviour stopping")    
    ui_client("stop")
    if moving == True:
        cancelled += 1
## Subcriber to know when a goal is reached
sub = rospy.Subscriber('/go_to_point/result',GoalReachingActionResult, clbk)
        
b1.on_click(on_button1_clicked)
b2.on_click(on_button2_clicked)

H2=HBox([b1, b2])
display(H2)

# This module implements two slider to set velocities

 The velocities are published on the topic **/cmd_vel_filter**, and then "elaborated" by the node **go_to_point.py** to set the velocities on **/cmd_vel**

In [ ]:
## Declaration for linear velocity slider
linear = widgets.FloatSlider(description='linear velocity', min=0, max=1.0)
display(linear)

## Declaration for angular velocity slider
angular = widgets.FloatSlider(description='angular velocity', min=-2.0, max=2.0)
display(angular)

## Publisher to /cmd_vel_filter to send a velocity to the robot
pub = rospy.Publisher('/cmd_vel_filter', Twist, queue_size=10)
msg = Twist()

## Function the updates the msg when the linear slider is used
#
#
def on_value_change(change):
    global msg
    msg.linear.x = change['new']
    
## Function the updates the msg when the angular slider is used
#
#    
def on_valueang_change(change):
    global msg
    msg.angular.z = change['new']

## Function triggered when the publish button is pressed
#
# Publishes on topic /cmd_vel_filter
def on_button8_clicked(b):
    global msg
    print("Setting velocities.....")
    pub.publish(msg)
    
linear.observe(on_value_change, names='value')
angular.observe(on_valueang_change, names='value')

## Button declaration
b8 = Button(
    description='Set velocities',
    layout=Layout(width='25%', align="center"),
    style=ButtonStyle(button_color='lightblue'))

b8.on_click(on_button8_clicked)

b8

# This module implements 5 buttons to directly control the robot

It publishes messages of type Twist on cmd_vel accordigly to 4 buttons that allow to go straight, go backwards, rotate left or right.
Also a button for stopping the robot is implemented.Every time a button is pressed the previously set goal is cancelled

In [ ]:
client = actionlib.SimpleActionClient('go_to_point', GoalReachingAction)

b3=Button(
    description='Backwards',
    layout=Layout(width='auto', grid_area='b3'),
    icon='fa-caret-down'
)

b4=Button(
    description='Turn Right',
    layout=Layout(width='auto', grid_area='b4'),    
    icon='fa-caret-right'
)

b5=Button(
    description='Turn Left',
    layout=Layout(width='auto', grid_area='b5'),    
    icon='fa-caret-left'
)

b6=Button(
    description='Forward',
    layout=Layout(width='auto', grid_area='b6'),
    icon='fa-caret-up'
)

b7=Button(
    description='Stop',
    layout=Layout(width='auto', grid_area='b7'),
    icon='fa-stop-circle'
)

def on_button3_clicked(b):
    client.cancel_goal
    msg.linear.x = -0.4
    msg.linear.y = 0.0
    msg.angular.z = 0.0
    pub.publish(msg)
    
def on_button4_clicked(b):
    client.cancel_goal
    msg.linear.x = 0.0
    msg.linear.y = 0.0
    msg.angular.z = 0.4
    pub.publish(msg)
    
def on_button5_clicked(b):
    client.cancel_goal    
    msg.linear.x = 0.0
    msg.linear.y = 0.0
    msg.angular.z = -0.4
    pub.publish(msg)
    
def on_button6_clicked(b):
    client.cancel_goal
    msg.linear.x = 0.4
    msg.linear.y = 0.0
    msg.angular.z = 0.0
    pub.publish(msg)    
    
def on_button7_clicked(b):
    client.cancel_goal
    msg.linear.x = 0.0
    msg.linear.y = 0.0
    msg.angular.z = 0.0
    pub.publish(msg)
    
b3.on_click(on_button3_clicked)
b4.on_click(on_button4_clicked)
b5.on_click(on_button5_clicked)
b6.on_click(on_button6_clicked)
b7.on_click(on_button7_clicked)

display(b7)

GridBox(children=[b3, b4, b5, b6],
layout=Layout(
width='60%',
grid_template_rows='auto auto',
grid_template_columns='33% 33% 33%',
grid_template_areas='''
" . b6 . "
"b5 b3 b4 "
''')
)

# This module implements two subscribers to /cmd_vel and /odom

 It retrieves and saves robot position, and robot linear and angular velocities accordingly to both /cmd_vel and /odom

In [ ]:
## Var to store x position of the robot
x_pose=[]
## Var to store y position of the robot
y_pose=[]
## Var to store orientation of the robot
z_orient=[]

## Callback of subscriber to /cmd_vel
#
# Stores the linear and angular velocities.
# @var cmd_lin_data used to store linear velocity information
# @var cmd_ang_data used to store angular velocity information
def cmd_vel_callback(msg):
    global cmd_lin_data,cmd_ang_data
    if len(cmd_lin_data)>10:
        cmd_lin_data.pop(0)
        cmd_ang_data.pop(0)
    #get values
    iupsilon1 = np.float32(msg.linear.x)
    iupsilon2 = np.float32(msg.angular.z)
    #update arrays
    cmd_ang_data.append(iupsilon2)
    cmd_lin_data.append(iupsilon1)

## Callback of subscriber to /odom
#
# Stores the linear and angular velocities and the pose of the robot
# @var cmd_lin_data used to store linear velocity information
# @var cmd_ang_data used to store angular velocity information
# @var x_pose store x position of the robot
# @var y_pose store y position of the robot
# @var z_orient store orientation of the robot
def act_vel_callback(msg):
    
    global time_passed, act_lin_data,act_ang_data
    global x_pose,y_pose,z_orient
    
    x_pose.append(msg.pose.pose.position.x)
    y_pose.append(msg.pose.pose.position.y)
    z_orient.append(msg.pose.pose.orientation.z)
    
    if len(time_passed)>10:
        time_passed.pop(0)
        act_lin_data.pop(0)
        act_ang_data.pop(0)
    #get angular and linear velocity
    y1 = np.float32(msg.twist.twist.angular.z)
    y2 = np.float32(msg.twist.twist.linear.x)
    #update timeline
    t = np.float32(time.time()-clock)
    #actual data inserted in array
    act_ang_data.append(y1)
    time_passed.append(t)
    act_lin_data.append(y2)
    

cmd_sub = rospy.Subscriber('/cmd_vel', Twist, cmd_vel_callback)
odom_sub = rospy.Subscriber('/odom', Odometry, act_vel_callback)

# This module sets up a plot for displaying linear velocities from /cmd_vel and /odom

The velocities displayed are the ones obtained and saved in the previous module


In [ ]:
def init1():
    line1.set_data([], [])
    line2.set_data([], [])
    return [line1,line2]

fig1, ax1 = plt.subplots()

line1, = ax1.plot([], [], color ='r', label ='/odom')
line2, = ax1.plot([], [], color ='b', label ='/cmd_vel')
ax1.set_xlabel('time (s)') 
ax1.set_ylabel('velocity m/s')
ax1.set_title("Linear velocity")
plt.legend(loc='upper right')

## The animate function and the animator object declaration.

This part actually updates the plot

In [ ]:
## Animate function
#
# Updates the plot accordingly to velocities and time
def animate1(i):
    #updated values are set in the line
    line1.set_data(time_passed, act_lin_data)
    line2.set_data(time_passed, cmd_lin_data)
    ax1.axis([time_passed[0]-0.2,time_passed[9]+0.2,-3,3]) #x axis is continously updated too   
    return [line1,line2]

#animator
anim1 = animation.FuncAnimation(fig1, animate1, init_func=init1,
                               frames=100, interval=20, blit=True)

# This module sets up a plot for displaying angular velocities from both /cmd_vel and /odom

The velocities displayed are the ones obtained and saved in module 5

In [ ]:
def init2():
    line3.set_data([], [])
    line4.set_data([], [])
    return [line3,line4]

fig2, ax2 = plt.subplots()

line3, = ax2.plot([], [], color ='r', label ='/odom')
line4, = ax2.plot([], [], color ='b', label ='/cmd_vel')
ax2.set_xlabel('time (s)') 
ax2.set_ylabel('velocity rad/s')
ax2.set_title("Angular velocity")
plt.legend(loc='upper right')
 

## The animate function and the animator object declaration.

This part actually updates the plot

In [ ]:
## Animate function
#
# Updates the plot accordingly to velocities and time
def animate2(i):
    line3.set_data(time_passed, act_ang_data)
    line4.set_data(time_passed, cmd_ang_data)
    ax2.axis([time_passed[0]-0.2,time_passed[9]+0.2,-3,3])
    return [line3,line4]

anim2 = animation.FuncAnimation(fig2, animate2, init_func=init2,
                               frames=100, interval=20, blit=True)   

# This module sets up a plot for displaying robot pose

The pose is the pose retrieved from **/odom** in module number 5

In [ ]:
fig_3, ax3 = plt.subplots()

ax3.set_xlim(( -10, 10))
ax3.set_ylim((-10, 10))

ax3.set_title('robot pose')

line, = ax3.plot([], [], 'ro')


def init3():
    line.set_data([], [])
    return (line)

## The animate function and the animator object declaration.

This part actually updates the plot

In [ ]:
def animate3(i):
    line.set_data(x_pose, y_pose)
    return (line)

anim3 = animation.FuncAnimation(fig_3, animate3, init_func=init3,
                               frames=100, interval=20, blit=True)

# This module sets up a plot for displaying reached vs cancelled targets

The values displayed are the ones updated and saved in module number 1, in which when the button to cancel the goal is pressed the variable **cancelled** gets incremented, and a subscriber to **/go_to_point/result** increments the value of variable **reached** every time a target position is achieved. Here the values are retrieved every time the button to update the plot is pressed, and the plot is redrawn accordingly.

In [ ]:
fig_4, ax4 = plt.subplots()
types = ['Reached', 'Cancelled']
ax4.set_title('Reached vs Aborted objectives')
ax4.set_ylabel('Scores')

## Function triggered when clicked on button
#
# Updates the plot
def on_button9_clicked(b):
    y=(reached, cancelled)
    ax4.bar(types,y, color = ['red', 'blue'])
    print("updated")

    
##Button declaration    
b9 = Button(
    description='Update plot',
    layout=Layout(width='25%', align="center"),
    style=ButtonStyle(button_color='lightblue'))

b9.on_click(on_button9_clicked)

b9


# This module sets up a plot for displaying elapsed times for rached targets

Here the values are retrieved every time the button to update the plot is pressed, and the plot is redrawn accordingly.

In [ ]:
n_bins = reached +1

fig_5, ax5 = plt.subplots()

ax5.set_title('Elapsed times for reached objectives')

## Function triggered when clicked on button
#
# Updates the plot
def on_button10_clicked(b):
    ax5.hist(times, bins=n_bins, color=['blue'])
    print("updated")
    
## Button declaration    
b10 = Button(
    description='Update plot',
    layout=Layout(width='25%', align="center"),
    style=ButtonStyle(button_color='lightblue'))

b10.on_click(on_button10_clicked)

b10
